<a href="https://colab.research.google.com/github/Ali-Bagherzadeh/Final-Exam/blob/main/Final_Exam_Assignment_ab4_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ali Bagherzadeh final Exam Assignment (40 points - Total of 45 is Possible)
## Due December 7, 2022, @ 8:00 ampip install keras
Note that there will be no extensions given for this assignment as there is a tight timeline for grading. 

For this assignment, I have provided each of you with your own training dataset. Your goal is to train a deep neural network to uncover the code image provided to you. 

I will provide you with instructions throughout. 

In [ ]:
# Add your import statements here
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import load
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
sns.set()
import urllib.request
from sklearn.model_selection import train_test_split
from scipy.special import expit as activation_function
from scipy.stats import truncnorm
from keras.models import Sequential
from keras.layers import Dense, Activation
import os
import sys
import time
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import Dataset
from torchvision import datasets

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from sklearn.model_selection import cross_val_score, train_test_split


In [ ]:
# This is a tool I have provided you to help you download your file.

def download_file(url, filename):
    """
    A function that downloads the data file from a URL
    Parameters
    ----------
    url : string
        url where the file to download is located
    filename : string
        location where to save the file
    reporthook : function
        callback to display the download progress
    """
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename, reporthook)
        
def reporthook(count, block_size, total_size):
    """
    A function that displays the status and speed of the download
    """

    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration + 0.0001))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

In [ ]:
# You can download your file by typing your first name into the name block
# The name used is the first part of your first name as listed in BB learn
# If you have problems downloading the data please reach out to me

name = 'Ali'
download_file(f'https://zenodo.org/record/7339649/files/data_{name}.npz?download=1','data.npz')

...100%, 24 MB, 3248 KB/s, 7 seconds passed

## Loading the Data (3 points)
The data is provided to you as a compressed NumPy array saved as 'data.npz'. When working with real data you might need to figure out how data is stored. Use the information on 'npz' files to figure out what data you have. The data file contains three NumPy arrays. 
1. The features for the training dataset
2. The regression values for the training dataset
3. The validation features that contain your code

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# Your Code goes here
dataAli = np.load('/content/gdrive/MyDrive/AliMl/data_Ali.npz')
lst = dataAli.files
for item in lst:
    print(item)
    print(dataAli[item])

training_feat
[[50.15881786  0.14740836 47.24435191 ... 19.23321463 21.46233564
   0.65416514]
 [76.35249032  1.83990513 22.14138396 ... 20.50781737 43.76897403
  45.89079564]
 [47.16151439  0.11889541 36.88076261 ... 30.8507654  45.18006429
  73.74330105]
 ...
 [62.39784079  1.52349286 39.20841027 ... 13.18610795 78.0266964
  51.64192514]
 [28.85380729  0.14590989 25.24930434 ...  6.59723584 79.35698998
  48.00198864]
 [41.18016932  1.1209152  23.94349425 ...  1.82253516 28.29859169
  58.92101681]]
training_true
[[0.79256891 0.37791841 0.35093268]
 [0.31450257 0.37992679 0.26829553]
 [0.31390719 0.60961793 0.58662354]
 ...
 [0.20300577 0.38135618 0.69663442]
 [0.38321709 0.28029869 0.64965628]
 [0.61621451 0.30044216 0.67955223]]
validation_feat
[[17.55877886  4.35091121 23.74038128 ... 14.97936909 23.89848764
  63.04844499]
 [17.55877886  4.35091121 23.74038128 ... 14.97936909 23.89848764
  63.04844499]
 [17.55877886  4.35091121 23.74038128 ... 14.97936909 23.89848764
  63.04844499]


In [ ]:
print('training_feat shape is:', dataAli['training_feat'].shape)
print('training_true shape is:',dataAli['training_true'].shape)
print('validation_feat shape is:',dataAli['validation_feat'].shape)

training_feat shape is: (100000, 30)
training_true shape is: (100000, 3)
validation_feat shape is: (65536, 30)


In [ ]:
training_feat = dataAli['training_feat']
training_true = dataAli['training_true']
validation_feat = dataAli['validation_feat']

## Preprocessing the Data (5 points)

You should explore the data and figure out the best way to preprocess the data. 

Hints: 
1. For the regression values, these at the end will represent colors in RGB space from [0,1]. It is recommended to use a max-min scalar between 0 and 1. 
2. For the training features, you should look at the data and determine the best scaling method. Look at our class notes for a reminder of what other scaler might be useful. 

In [ ]:
# Your code goes here
# Calculate and print minimum, maximum, mean, median and std values
print("minimum training_feat: %.3f" % np.min(training_feat))
print("maximum training_feat:%.3f" % np.max(training_feat))
print("mean training_feat:%.3f" % training_feat.mean())
print("median training_feat:%.3f" % np.median(training_feat))
print("std training_feat:%.3f" % np.std(training_feat))

minimum training_feat: 0.000
maximum training_feat:94.941
mean training_feat:25.740
median training_feat:19.659
std training_feat:21.234


In [ ]:
print("minimum training_true:%.3f" % np.min(training_true))
print("maximum training_true:%.3f" % np.max(training_true))
print("mean training_true:%.3f" % training_true.mean())
print("median training_true:%.3f" % np.median(training_true))
print("std training_true:%.3f" % np.std(training_true))

minimum training_true:0.000
maximum training_true:1.000
mean training_true:0.492
median training_true:0.491
std training_true:0.159


In [ ]:
y_scaler = MinMaxScaler()
scaled_training_true = y_scaler.fit_transform(training_true)
x_scaler = StandardScaler()
scaled_training_feat = x_scaler.fit_transform(training_feat)
scaled_validation_feat = x_scaler.transform(validation_feat)

## Building the Dataset (5 points)

When training neural networks it is important to build a dataset that allows the machinery to sample the data. This also can be used to conduct some preprocessing of the data to make it work with PyTorch. 

I have provided you with the framework for a Dataset Class. 

You should:
1. Convert the x and y data to a tensor 'float32' and put it on the GPU.
2. Save the len of the data
3. Add the code so when `__getitem__` is called it returns the x and y values
3. make it so `__len__` returns the lenght when calle


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
class Data(Dataset):
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, y: np.ndarray, device = 'cuda') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X.astype(np.float32)).to(device)
    self.y = torch.from_numpy(y.astype(np.float32)).to(device)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index], self.y[index]

  def __len__(self) -> int:
    return self.len

## Train-test Split (3 points)

1. You should conduct a train-test split of the training data so you can make sure that your model does not overfit the data. A good ratio is 66/33 train 
2. You should instantiate the training dataset using the data class implemented above.


In [ ]:
dataclass = Data(scaled_training_feat, scaled_training_true)

In [ ]:
# data visualization after class transformation for first element
features1, labels1 = dataclass[1]
print(features1, labels1)

tensor([ 1.4874, -0.7037, -0.4346, -1.5573, -0.8812, -1.3585, -1.4127, -0.0666,
         0.5727,  0.2660,  1.4100, -0.2015, -0.8363, -0.9285,  1.1009, -1.1602,
         0.7519, -0.1965, -0.8376,  1.6895, -1.1739,  0.9670,  0.3658, -1.2774,
         1.0545,  0.8572, -1.7039,  0.2780, -0.0160,  0.1212], device='cuda:0') tensor([0.3145, 0.3799, 0.2683], device='cuda:0')


In [ ]:
# Your code goes here
X_train,X_test,y_train,y_test=train_test_split(dataclass.X, dataclass.y,test_size=0.3,random_state=3)
print(f'Test size X and y: {len(X_test)} , {len(y_test)}')
print(f'Train size X and y: {len(X_train)} , {len(y_train)}')

Test size X and y: 30000 , 30000
Train size X and y: 70000 , 70000


## Build the Dataloader (3 points)

Pytorch uses DataLoaders to efficiently sample from a training dataset. Instantiate a Pytorch DataLoader using the dataset. 

You should set the following parameters:
1. Batch size = 64
2. Shuffle = True

In [ ]:
# Your code goes here
batch_size = 64

# Create data loaders.
train_dataloader_X = DataLoader(X_train, batch_size=batch_size, shuffle=True)
train_dataloader_y = DataLoader(y_train, batch_size=batch_size, shuffle=True)
test_dataloader_X = DataLoader(X_test, batch_size=batch_size, shuffle=True)
test_dataloader_y = DataLoader(y_test, batch_size=batch_size, shuffle=True)

In [ ]:
print(f'train_dataloader_X and train_dataloader_y: {len(train_dataloader_X)} , {len(train_dataloader_y)}')
print(f'test_dataloader_X and test_dataloader_y: {len(test_dataloader_X)} , {len(test_dataloader_y)}')

train_dataloader_X and train_dataloader_y: 1094 , 1094
test_dataloader_X and test_dataloader_y: 469 , 469


In [ ]:
next(iter(train_dataloader_X))

tensor([[ 0.7384, -0.6274,  0.9602,  ...,  0.7857,  0.0905, -1.4873],
        [-1.0121, -0.4975,  1.3440,  ..., -0.5553,  0.0862,  1.2823],
        [ 1.2869, -0.4691, -1.5108,  ...,  0.3584,  1.5616, -1.0055],
        ...,
        [ 1.4791, -0.9520, -1.5834,  ...,  0.9941, -1.4693,  0.0837],
        [ 1.3195, -0.0480, -0.5547,  ..., -0.0141,  1.0717,  0.4967],
        [-1.0702,  1.5034,  0.9880,  ...,  1.3588,  1.6577, -1.1785]],
       device='cuda:0')

## Building a Neural Network (5 points)

Using the provided class framework which inherits the `nn.Module` type in PyTorch builds a 4-layer neural network to complete the multiple regression.



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Neural_Network(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
        super(Neural_Network, self).__init__()
        # in this part you should intantiate each of the layer components
        self.fc1 = nn.Linear(30, 30)
        self.fc2 = nn.Linear(30, 30)
        self.fc3 = nn.Linear(30, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
        #return F.log_softmax(x, dim=1)

## Instantiate the Model (3 points)



In [ ]:
# number of features (len of X cols)
input_dim = len(train_dataloader_X)
# number of hidden layers set this to 50
hidden_layers = 50
# Add the number of output dimensions
output_dim = 3

In [ ]:
# initiate the regression model
# make sure to put it on your GPU
model = Neural_Network(input_dim, hidden_layers, output_dim).to(device)
print(model)

Neural_Network(
  (fc1): Linear(in_features=30, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)


In [ ]:
# criterion to computes the loss between input and target
# Choose a good criteria
# Initialize the loss function
loss_fn = nn.MSELoss()

# optimizer that will be used to update weights and biases
# you can choose any optimizer. I would recommend ADAM.
# This problem should not be hard to optimize. A good starting learning rate is 3e-5. 
learning_rate = 3e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# https://courses.coe.drexel.edu/MEM/MEMT680/Topic_10/7_optimization_tutorial.html?highlight=adam

## Train the Model (5 points)

Training the model is conducted in a number of steps using loops.

1. Set up a loop for each epoch
2. Set a parameter to save the running loss
3. Set up a nested loop that goes through the batches from the DataLoader you built
    - I would recommend using enumerate to include the counts in the loop
    - The dataloader will return a tuple that is the inputs and the labels
4. Conduct the forward propagation of the model
    - Give the model the inputs and compute the outputs
    - Compute the loss given the criteria. 
5. Use the zero gradient method to remove the gradients from the optimizer
6. Use the backward method to compute the gradients
7. Use the step method in the optimizer to take an optimization step
8. Compute the running loss by calling the item method and adding it to the running loss for each minibatch
9. For each epoch print the epoch and the loss

Note: If you find this challenging I would recommend that you look at examples of other pytorch training loops online. This is a very standard workflow. 

In [ ]:
from torch.utils.data import Dataset, TensorDataset

tensor_Xp_train = torch.stack([torch.Tensor(el) for el in X_train])
tensor_yp_train = torch.stack([torch.Tensor(el) for el in y_train])
dataset_p_train = TensorDataset(tensor_Xp_train, tensor_yp_train)
loader_p_train = DataLoader(dataset_p_train, batch_size=batch_size, shuffle=True)

tensor_Xp_test = torch.stack([torch.Tensor(el) for el in X_test])
tensor_yp_test = torch.stack([torch.Tensor(el) for el in y_test])
dataset_p_test = TensorDataset(tensor_Xp_test, tensor_yp_test)
loader_p_test = DataLoader(dataset_p_test, batch_size=batch_size, shuffle=True)

In [ ]:
epochs = 20

# sets the number of epochs to train 20 should be sufficent.
# This should take about 5-10 minutes to train.

# Your code should go here
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_p_train, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.362271  [    0/70000]
loss: 0.325181  [ 6400/70000]
loss: 0.285085  [12800/70000]
loss: 0.249203  [19200/70000]
loss: 0.222825  [25600/70000]
loss: 0.181805  [32000/70000]
loss: 0.164308  [38400/70000]
loss: 0.134513  [44800/70000]
loss: 0.107801  [51200/70000]
loss: 0.088113  [57600/70000]
loss: 0.078435  [64000/70000]
Epoch 2
-------------------------------
loss: 0.045102  [    0/70000]
loss: 0.056662  [ 6400/70000]
loss: 0.036779  [12800/70000]
loss: 0.028278  [19200/70000]
loss: 0.032274  [25600/70000]
loss: 0.029257  [32000/70000]
loss: 0.034232  [38400/70000]
loss: 0.023014  [44800/70000]
loss: 0.023902  [51200/70000]
loss: 0.019381  [57600/70000]
loss: 0.020263  [64000/70000]
Epoch 3
-------------------------------
loss: 0.019835  [    0/70000]
loss: 0.024893  [ 6400/70000]
loss: 0.020631  [12800/70000]
loss: 0.021068  [19200/70000]
loss: 0.022485  [25600/70000]
loss: 0.017652  [32000/70000]
loss: 0.017531  [38400/70000]
loss: 0.02

In [ ]:
# https://www.google.com/search?q=pytorch+training+loops+online&rlz=1C1GCEB_enUS979US979&oq=pytorch+training+loops+online&aqs=chrome..69i57j33i299.949j0j7&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:86f4071d,vid:c36lUUr864M

# https://pyimagesearch.com/2021/07/12/intro-to-pytorch-training-your-first-neural-network-using-pytorch/

# https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans


## Validate the Model (3 points)

Use the test dataset from the train-test split to make sure your model is not overfitting

1. You can build a dataloader as you did before, this time with the test data.
2. Build a validation loop, which you should use `with torch.no_grad()` to make sure you do not modify the gradients, or weights. This will fix your model. 
3. Instantiate the loss to be 0.
4. Build a similar loop to grab the validation dataset. 
5. Compute the predictions with the model.
6. Compute the loss using your loss criteria.
7. Print the final loss determined.

In [ ]:
# Your code goes here
# https://courses.coe.drexel.edu/MEM/MEMT680/Topic_10/6_autogradqs_tutorial.html?highlight=torch%20no_grad

In [ ]:
# Your code should go here
def test_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test_loop(loader_p_test, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.005573  [    0/30000]
loss: 0.005347  [ 6400/30000]
loss: 0.006257  [12800/30000]
loss: 0.004829  [19200/30000]
loss: 0.006260  [25600/30000]
Epoch 2
-------------------------------
loss: 0.005391  [    0/30000]
loss: 0.004513  [ 6400/30000]
loss: 0.005964  [12800/30000]
loss: 0.005585  [19200/30000]
loss: 0.004781  [25600/30000]
Epoch 3
-------------------------------
loss: 0.006058  [    0/30000]
loss: 0.006891  [ 6400/30000]
loss: 0.005179  [12800/30000]
loss: 0.004359  [19200/30000]
loss: 0.006680  [25600/30000]
Epoch 4
-------------------------------
loss: 0.004959  [    0/30000]
loss: 0.005233  [ 6400/30000]
loss: 0.003994  [12800/30000]
loss: 0.005127  [19200/30000]
loss: 0.005712  [25600/30000]
Epoch 5
-------------------------------
loss: 0.004768  [    0/30000]
loss: 0.005449  [ 6400/30000]
loss: 0.005018  [12800/30000]
loss: 0.004683  [19200/30000]
loss: 0.004621  [25600/30000]
Epoch 6
-------------------------------
loss: 0.00

<p style="color:blue"> Question: Is your model overfitting or not? How do you know? (3 points) </p>

If the loss of the training set is much smaller than the loss of the test set, it means the model overfitting and fitted perfectly on the training set and does not work properly on the test set.

Type your response here

## Crack Your Code (3 points)

1. You can build a dataloader as you did before, this time with the validation features to view your code.
2. Build a loop, you should use `with torch.no_grad()` to make sure you do not modify the gradients or weights. This will fix your model. 
3. Compute the predictions of your model. 
    - Make sure you do all the same preprocess, the data has the same datatype, and is on the same device

In [ ]:
# Your code goes here
validation_dataloader_y = DataLoader(scaled_validation_feat, batch_size=batch_size, shuffle=True)

In [ ]:
def validation_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    with torch.no_grad():
      for X in dataloader:
        pred = model(X)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    validation_loop(validation_dataloader_y, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: ignored

## Reveal Your Code (3 points)

Your code is an image. there are (65536, 3) predictions this corresponds to a (256,256,3) RGB image. 
1. Use the detach() method to remove the gradients from the tensor
2. Transfer the tensor back to the 'cpu' if you had it on a GPU
3. Reshape the image into a 256,256,3 array. 
4. Plot your successful result.

In [ ]:
# Your Code goes here